In [2]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# 가상의 대규모 데이터셋 생성
np.random.seed(0)
x_train = np.random.uniform(-10, 10, size=1000)
y_train = 2.0 * x_train + 1.0 + np.random.normal(0, 2, size=1000)

# 그래프 초기화
tf.reset_default_graph()

# 플레이스홀더 설정
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

# 모델 가중치와 바이어스
W1 = tf.Variable(tf.random_normal([1, 10]), name='W1')
b1 = tf.Variable(tf.random_normal([10]), name='b1')
W2 = tf.Variable(tf.random_normal([10, 10]), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')
W3 = tf.Variable(tf.random_normal([10, 1]), name='W3')
b3 = tf.Variable(tf.random_normal([1]), name='b3')

# 모델 정의
layer_1 = tf.nn.relu(tf.add(tf.matmul(tf.expand_dims(X, -1), W1), b1))
layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, W2), b2))
predictions = tf.add(tf.matmul(layer_2, W3), b3, name='predictions')

# 손실 함수 및 옵티마이저 정의
loss = tf.reduce_mean(tf.square(predictions - Y))
optimizer = tf.train.AdamOptimizer().minimize(loss, name='train')  # 학습 연산 추가

# 세션 초기화 및 변수 초기화
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # 모델 학습
    epochs = 100
    for epoch in range(epochs):
        _, l = sess.run([optimizer, loss], feed_dict={X: x_train, Y: y_train})
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}, Loss: {l}')

    # 그래프 및 가중치 저장
    graph_def = sess.graph.as_graph_def()

    # 모든 변수와 그 값을 불러오기
    output_graph_def = tf.graph_util.convert_variables_to_constants(
        sess, graph_def, ['predictions'])

    # 모델을 .pb 파일로 저장
    with tf.gfile.GFile('model.pb', 'wb') as f:
        f.write(output_graph_def.SerializeToString())

    # 로드한 모델로 예측 (예시로 10개의 데이터를 예측)
    predictions_val = sess.run(predictions, feed_dict={X: x_train[:10]})
    print(predictions_val)


Epoch 10, Loss: 546.912109375
Epoch 20, Loss: 473.8365173339844
Epoch 30, Loss: 411.0013122558594
Epoch 40, Loss: 357.8159484863281
Epoch 50, Loss: 313.2666931152344
Epoch 60, Loss: 276.23468017578125
Epoch 70, Loss: 245.72311401367188
Epoch 80, Loss: 220.85906982421875
Epoch 90, Loss: 200.85867309570312
Epoch 100, Loss: 184.9615936279297
[[ -3.542549 ]
 [ -6.1069574]
 [ -2.9671807]
 [ -3.5784156]
 [ -2.817759 ]
 [ -3.826483 ]
 [ -3.0745263]
 [-11.372278 ]
 [-13.481796 ]
 [ -1.8846102]]
